# *KNN*算法

## 概述

k-近邻算法 $(k-Nearest Neighbour algorithm)$，又称为 *KNN* 算法，是数据挖掘技术中原理最简单
的算法。*KNN* 的工作原理：给定一个已知标签类别的训练数据集，输入没有标签的新数据后，在训练数
据集中找到与新数据最邻近的k个实例，如果这 $k$ 个实例的多数属于某个类别，那么新数据就属于这个类
别。可以简单理解为：由那些离 $X$ 最近的 $k$ 个点来投票决定 $X$ 归为哪一类。
使用欧式距离（欧几里得度量）来刻画两个样本点之间的距离
$$
dist(x,y) = \sqrt{(x_1-y_1)^{2} + (x_2-y_2)^{2} + \cdots + (x_n-y_n)^{2}} = \sqrt{\sum_{i=1}^{n}(x_i-y_i)^{2}}
$$

k-近邻算法步骤如下
1. 计算已知类别数据集合中的点与当前点之间的距离
2. 按照距离递增次序排序
3. 选取与当前点距离最小的 $k$ 个点
4. 确定前 $k$ 个点所在类别的出现频率
5. 返回前 $k$ 个点出现频率最高的类别作为当前点的预测类别

## *Python* 实现

### 算法实现

通过一个简单的例子了解 *KNN* 算法的工作原理：

1. 构建已经分好类的原始数据集
使用字典 `dict` 构建数据集，再将其转化成 $DataFrame$ 格式

In [10]:
import pandas as pd
row_data={'电影名称':['无问西东','后来的我们','前任3','红海行动','唐人街探案','战狼2'],
        '打斗镜头':[1,5,12,108,112,115],
        '接吻镜头':[101,89,97,5,9,8],
        '电影类型':['爱情片','爱情片','爱情片','动作片','动作片','动作片']}
movie_data = pd.DataFrame(row_data)
movie_data

,电影名称,打斗镜头,接吻镜头,电影类型
0,无问西东,1,101,爱情片
1,后来的我们,5,89,爱情片
2,前任3,12,97,爱情片
3,红海行动,108,5,动作片
4,唐人街探案,112,9,动作片
5,战狼2,115,8,动作片


2. 计算已知类别数据集中的点与当前点之间的距离

In [11]:
'''
iloc[: , :]
参数分别为行和列
左闭右开原则 
'''
new_data = [24,67] #待归类的样本点
dist = list((((movie_data.iloc[:6,1:3]-new_data)**2).sum(1))**0.5) #sum(1) 求数组每一行的和
dist

[41.048751503547585,
 29.068883707497267,
 32.31098884280702,
 104.4030650891055,
 105.39449701004318,
 108.45275469069469]

3. 将所有距离以上所有距离升序排序，然后选取距离最小的 $k$ 个点

这里选取 $k$ 的值为 $4$

In [12]:
dist_list = pd.DataFrame({'dist': dist, 'labels': (movie_data.iloc[:6, 3])})
dis_res = dist_list.sort_values(by = 'dist')[: 4]
dis_res

,dist,labels
1,29.068884,爱情片
2,32.310989,爱情片
0,41.048752,爱情片
3,104.403065,动作片


4. 确定前 $k$ 个点所在类别的出现频率

In [13]:
res = dis_res.loc[:,'labels'].value_counts()
res

爱情片    3
动作片    1
Name: labels, dtype: int64

5. 选择频率最高的类别作为当前待预测样本点的预测类别

In [14]:
new_data_label = []
new_data_label.append(res.index[0])
new_data_label

['爱情片']

### 函数封装

In [15]:
import pandas as pd

def KNN_classify(inX, dataSet, k):
    """KNN分类器

    Parameters
    ----------
    inX : 
        待分类的样本点
    dataSet : 
        _训练集（已知分类标签）
    k : 
        KNN参数 k
    """

    inX_label = []
    dist = list((((dataSet.iloc[:,1:3]-inX)**2).sum(1))**0.5)
    dist_list = pd.DataFrame({'dist': dist, 'labels': (dataSet.iloc[:, 3])})
    dis_res = dist_list.sort_values(by = 'dist')[: k]
    res = dis_res.loc[:,'labels'].value_counts()
    inX_label.append(res.index[0])
    return inX_label

In [16]:
# 测试函数运行结果
KNN_classify(new_data, movie_data, 3)

['爱情片']

### 数据归一化
将不同特征转换为无量纲的特征向量，避免个别特征对分类结果影响远远大于其他特征对分类的影响
最简单的 0-1 标准化公式如下：
$$
x_{normalization} = \frac{x-\min(x)}{\max(x)-\min(x)}
$$

In [17]:
def minmax_norm(dataSet):
    """minmax归一化处理

    Parameters
    ----------
    dataSet :
        原始数据集
    """    
    minDf = dataSet.min()
    maxDf = dataSet.max()
    normSet = (dataSet - minDf )/(maxDf - minDf)
    return normSet

### 划分训练集和测试集
为了测试分类效果，将原始数据集分成训练集和测试集两部分

一般随机选取 $90%$ 作为训练集，剩下的 10% 作为测试集

In [18]:
def randSplit(dataset, rate = 0.9):
    """随机划分训练集和测试集

    Parameters
    ----------
    dataset : 
        原始数据集
    rate :
        训练集划分比例
    """
    n = dataset.shape[0]
    m = int(n * rate)
    trainSet = dataset.iloc[:m, :]
    testSet = dataset.iloc[m:, :]
    testSet.index = range(testSet.shape[0])
    return trainSet, testSet

# 说明：该函数只适用于随机的原数据集

## 总结
- 优点
  - 算法简单，精度高，既可以做分类也可以用作回归
  - 可用于数值型和离散型数据
  - 无数据输入假定
  - 适合对稀有事件进行分类
- 缺点
  - 计算复杂性和空间复杂性较高
  - 计算量大，一般数值很大的时候不使用 $KNN$，但是单个样本又不能太少，否则容易发生误分
  - 样本不平衡问题
  - 可理解性较差，无法给出数据内在含义